In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, lr_scheduler

from avalanche.benchmarks.classic import SplitMNIST
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.models import SimpleMLP
from avalanche.training.supervised import Naive

seed = 1234
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size_train = 32
batch_size_test = 32
lr = 1e-3
momentum = 0.8

tasks = 10


/home/thealch3mist/University/Thesis/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
benchmark_instance = SplitMNIST(n_experiences=tasks,seed=1)

train_stream = benchmark_instance.train_stream
test_stream = benchmark_instance.test_stream

print(train_stream)
print(test_stream)
for idx, experience in enumerate(train_stream):
    dataset = experience.dataset

    print(f"Train dataset contains {len(dataset)} samples")

    ii = 0
    for x,y,t in dataset:
        print(x.shape, y, t)
        ii += 1
        if ii == 5:
            print("...")
            break

    test_expereince = test_stream[idx]
    cumulative_test = test_stream[:idx+1]

Train dataset contains 5421 samples
torch.Size([1, 28, 28]) 5 0
torch.Size([1, 28, 28]) 5 0
torch.Size([1, 28, 28]) 5 0
torch.Size([1, 28, 28]) 5 0
torch.Size([1, 28, 28]) 5 0
...
Train dataset contains 5918 samples
torch.Size([1, 28, 28]) 6 0
torch.Size([1, 28, 28]) 6 0
torch.Size([1, 28, 28]) 6 0
torch.Size([1, 28, 28]) 6 0
torch.Size([1, 28, 28]) 6 0
...
Train dataset contains 6742 samples
torch.Size([1, 28, 28]) 1 0
torch.Size([1, 28, 28]) 1 0
torch.Size([1, 28, 28]) 1 0
torch.Size([1, 28, 28]) 1 0
torch.Size([1, 28, 28]) 1 0
...
Train dataset contains 5958 samples
torch.Size([1, 28, 28]) 2 0
torch.Size([1, 28, 28]) 2 0
torch.Size([1, 28, 28]) 2 0
torch.Size([1, 28, 28]) 2 0
torch.Size([1, 28, 28]) 2 0
...
Train dataset contains 5923 samples
torch.Size([1, 28, 28]) 0 0
torch.Size([1, 28, 28]) 0 0
torch.Size([1, 28, 28]) 0 0
torch.Size([1, 28, 28]) 0 0
torch.Size([1, 28, 28]) 0 0
...
Train dataset contains 5851 samples
torch.Size([1, 28, 28]) 8 0
torch.Size([1, 28, 28]) 8 0
torch.Si

In [12]:
benchmark_instance.streams['train'].size

AttributeError: 'NCStream' object has no attribute 'shape'

In [8]:
from Scaled_KAN import FastKAN
# model
model = Fast_KAN(hidden_layers=64, num_classes=10)

# CL Benchmark Creation
perm_mnist = PermutedMNIST(n_experiences=3)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing

optimizer = Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=momentum)

criterion = CrossEntropyLoss()
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, epoch_running=True, 
                     experience=True, stream=True))

# Continual learning strategy
cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2, 
    eval_mb_size=32, evaluator=eval_plugin, device=device)

# train and test loop

results = []
for train_task in train_stream:
    cl_strategy.train(train_task, num_workers=4)
    results.append(cl_strategy.eval(test_stream))

NameError: name 'Fast_KAN' is not defined